# Генеративные состязательные сети (GAN - Generative Adversarial Networks)

Все приложения нейронных сетей, которые мы исследовали ранее, были **дискриминативными моделями**, которые принимают входные данные и обучаются формировать выход с определенными метками.  В этом блокноте  создадим **генеративные модели**, используя нейронные сети. В частности, мы узнаем, как создавать модели, которые генерируют новые образы, напоминающие образы из наборов тренировочных данных.

### Что такое  GAN?

В 2014 году [Goodfellow и др.](https://arxiv.org/abs/1406.2661) представили метод обучения генеративных моделей, которые называются Генеративными состязательными сетями (сокращенно GAN). В GAN мы строим две отдельные нейронные сети. Первая сеть - это традиционная классификационная сеть, называемая **дискриминатором**. Мы обучаем дискриминатор воспринимать изображения и классифицировать их как реальные (принадлежащие обучающему набору) или поддельные (не представленные в обучающем наборе). Вторая сеть, называемая **генератором**, принимает случайный шум в качестве входного сигнала и преобразовывает его, используя нейронную сеть, в изображение. Цель генератора - обмануть дискриминатор, заставив его "думать", что сгенерированные изображения реальны.

Мы можем представить себе этот процесс применения генератора ($G$), пытающегося обмануть дискриминатор  ($D$), 
и дискриминатора, пытающегося правильно классифицировать реальные и фальшивые образы как минимаксную игру:
$$\underset{G}{\text{minimize}}\; \underset{D}{\text{maximize}}\; \mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] + \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$
где $x \sim p_\text{data}$  - выборки из входных данных, $z \sim p(z)$ - выборки случайных шумов, $G(z)$ - сгенерированные изображения с использованием генератора нейронной сети $G$, и $D$ - это выходной сигнал дискриминатора, определяющий вероятность того, что вход является реальным. В [Goodfellow и др.](https://arxiv.org/abs/1406.2661)  анализируется эта минимаксная игра и демонстрируется как она связана с минимизацией дивергенции Дженсена-Шеннона между распределением обучающих данных и сгенерированными выборками из $G$.

Чтобы оптимизировать эту минимаксную игру, будем выполнять выбор между шагами градиентного *спуска* по целевой функции для $G$ и шагами  градиентного *подъема* по целевой функции для $D$:
1. обновить **генератор** ($G$), чтобы минимизировать вероятность того, что __дискриминатор сделает правильный выбор__.
2. обновить **дискриминатор** ($D$), чтобы максимизировать вероятность того, что __дискриминатор сделает правильный выбор__.

Хотя эти обновления полезны при анализе, они не очень хорошо работают на практике. Вместо этого мы будем использовать другую цель при обновлении генератора: максимизировать вероятность того, что **дискриминатор сделает неправильный выбор**. Это небольшое изменение помогает снизить остроту проблемы с исчезновением градиента генератора, когда дискриминатор уверен в своем решении. Это стандартное обновление, используется в большинстве работ по GAN , оно также использовалось в оригинальной статье [Goodfellow et al.](https://arxiv.org/abs/1406.2661).


В этом задании мы будем чередовать следующие шаги обновления параметров нейросетей:
1. Обновить генератор ($G$) , чтобы максимизировать вероятность того, что дискриминатор сделает неправильный выбор на сгенерированных данных:
$$\underset{G}{\text{maximize}}\;  \mathbb{E}_{z \sim p(z)}\left[\log D(G(z))\right]$$
2. Обновить дискриминатор ($D$), чтобы максимизировать вероятность того, что дискриминатор сделает правильный выбор на реальных и сгенерированных данных
$$\underset{D}{\text{maximize}}\; \mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] + \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$



### Что ещё?

С 2014 года сети GAN превратились в огромную область исследований с  [сотнями новых статей](https://github.com/hindupuravinash/the-gan-zoo) . GAN являются одними из самых сложных и привередливых моделей для обучения (см.[github repo](https://github.com/soumith/ganhacks), который содержит набор из 17 хаков, которые полезны для работы с моделями). Улучшение стабильности и надежности обучения GAN - открытый вопрос исследования, каждый день появляются новые статьи! Классический учебный материал по GAN см. [здесь](https://arxiv.org/abs/1701.00160). Также есть  недавние захватывающие работы, которые заменяют целевую функцию на расстояние Вассерштейна и дают гораздо более стабильные результаты для следующих архитектур моделей: [WGAN](https://arxiv.org/abs/1701.07875), [WGAN-GP](https://arxiv.org/abs/1704.00028).

GAN не единственный способ тренировать генеративную модель! Другие подходы к генеративному моделированию можно найти в [книге](http://www.deeplearningbook.org/contents/generative_models.html) .  Другой популярный способ обучения нейронных сетей как генеративных моделей - это вариационные автоэнкодеры (совместно предложенные [здесь](https://arxiv.org/abs/1312.6114) и [здесь](https://arxiv.org/abs/1401.4082)). Вариационные автоэнкодеры объединяют нейронные сети с вариационным выводом для обучения глубоких генеративных моделей. Эти модели имеют тенденцию быть намного более стабильными и более простыми в обучении, но в настоящее время они не формируют образы, которые столь же качественны, как GAN.

Ниже приведены примеры изображений, которые вы должны ожидать в качестве результатов выполнения этого блокнота (ваши могут выглядеть немного иначе):

![caption](gan_outputs_tf.png)


## Установки

In [ ]:
import tensorflow as tf
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Набор всмогательных утилит

def show_images(images):
    """ Отображает изображения в клеточной структуре"""
    images = np.reshape(images, [images.shape[0], -1])  #реформатирование изображений к (batch_size, D)
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return

def preprocess_img(x):
    return 2 * x - 1.0

def deprocess_img(x):
    return (x + 1.0) / 2.0

def rel_error(x,y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def count_params(model):
    """Возвращает число параметров в текущем TensorFlow графе """
    param_count = np.sum([np.prod(p.shape) for p in model.weights])
    return param_count

answers = np.load('gan-checks-tf.npz')

NOISE_DIM = 96

## Набор данных
 
Общеизвестно, что GAN привередливы к значениям гиперпараметров, а также требуют многих тренировочных эпох. Чтобы сделать возможным выполнение этого задания  без графического процессора, мы будем работать с набором данных MNIST, который состоит из 60 000 обучающих и 10 000 тестовых изображений. Каждое изображение содержит центрированное изображение белой цифры на черном фоне (от 0 до 9). Это был один из первых наборов данных, использованных для обучения сверточных нейронных сетей, и он довольно прост - стандартная модель CNN легко классифицирует данные этого набора с точностью 99%.
 

**Внимание**: класс MNIST возвращает изображения как векторы формы (размер блока, 784). Если Вы хотите обрабатывать их как изображения, то необходимо привести их к форме (размер блока, 28,28) или (размер блока, 28,28,1). Тип элементов np.float32, а значения ограничены интервалом [0,1].

In [ ]:
# Определим вспомогательную функцию load_data для загрузки набора MNIST
# Необходимо, чтобы набор данных mnist.npz
# находился в текущей папке блокнота

def load_data(path):
    with np.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)

# Пример обращения к функции
#(x_train, y_train), (x_test, y_test) = load_data('mnist.npz')

In [ ]:
class MNIST(object):
    def __init__(self, batch_size, shuffle=False):
        """
        Создает итератор по данным MNIST
        
         Входы:
         - batch_size: целое число, задающее количество элементов в мини-блоке
         - shuffle: (необязательный) тип Boolean, следует ли перемешивать данные на каждой эпохе
        """
        # Загрузка данных с помощью определенной выше функции load_data
        train, _ = load_data('mnist.npz')
        # Либо можно воспользоваться возможностями tf.keras.datasets
        # train, _ = tf.keras.datasets.mnist.load_data()
        
        X, y = train
        X = X.astype(np.float32)/255
        X = X.reshape((X.shape[0], -1))
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B)) 

In [ ]:
# отображение миниблока изображений
mnist = MNIST(batch_size=16) 
show_images(mnist.X[:16])

## Функция активации: ReLU с утечкой (LeakyReLU)
В приведенной ниже ячейке блокнота Вы должны реализовать LeakyReLU. См.   уравнение (3) в [статье](http://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf). LeakyReLU предохраняет  ReLU нейроны от "умирания" и они часто используются в  GAN (как и блоки maxout, при этом увеличивается размер модели).

СОВЕТ: Вы должны использовать `tf.maximum`

In [ ]:
def leaky_relu(x, alpha=0.01):
    """    
     Вычисление функции активации  ReLU с утечкой.
    
     Входы:
     - x: Тензор TensorFlow  с произвольной формой
     - alpha: параметр утечки для  ReLU с утечкой.
    
     Возвращает:
     Тензор той же формы, что и x
    """
    # Задание: реализовать ReLU с утечкой
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
            
    pass
    return 
    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****



Протестируйте Вашу реализацию leaky ReLU. Ошибки должны быть < 1e-10

In [ ]:
def test_leaky_relu(x, y_true):
    y = leaky_relu(tf.constant(x))
    print('Максимум ошибки: %g'%rel_error(y_true, y))

test_leaky_relu(answers['lrelu_x'], answers['lrelu_y'])

## Случайный шум

Сгенерируйте `Tensor` TensorFlow, содержащий равномерно распределенный шум от -1 до 1, размерность тензора `[batch_size, dim]`.

In [ ]:
def sample_noise(batch_size, dim):
    """
     Генерирует случайный равномерный шум в диапазоне от -1 до 1.
  
     Входы:
     - batch_size: целое число, задающее размер миниблока данных шума
     - dim: целое число, определяющее длину вектора шума
    
     Возвращает:
     Тензор TensorFlow , содержащий равномерный шум в [-1, 1] с формой [batch_size, dim]
    """
    # Задание: сделать выборку и вернуть шум
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
            
    pass
    return 
    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****



Убедитесь, что шум имеет правильный размер и тип:

In [ ]:
def test_sample_noise():
    batch_size = 3
    dim = 4
    z = sample_noise(batch_size, dim)
    # Проверьте, что z имеет правильную форму
    assert z.get_shape().as_list() == [batch_size, dim]
    # Проверьте, что z тензор, а не  numpy массив
    assert isinstance(z, tf.Tensor)
    # Проверьте, что мы получаем различный шум в разных попытках
    z1 = sample_noise(batch_size, dim)
    z2 = sample_noise(batch_size, dim)
    assert not np.array_equal(z1, z2)
    # Проверьте, что шум находится в заданном диапазоне
    assert np.all(z1 >= -1.0) and np.all(z1 <= 1.0)
    print("Все тесты пройдены!")
    
test_sample_noise()

## Дискриминатор

Выполним первый шаг - построим дискриминатор. Подсказка: Вы должны использовать слои из `tf.keras.layers` для построения модели. Все полносвязанные слои должны содержать  смещения. Для инициализации  используйте инициализатор по умолчанию, реализуемый функциями `tf.keras.layers`.

Архитектура в порядке следования слоев:

     * Полносвязанный слой с размерами: вход - 784 и выход - 256
     * LeakyReLU с альфа 0,01
     * Полносвязанный слой с выходным размером 256
     * LeakyReLU с альфа 0,01
     * Полносвязанный слой с выходным размером 1

Таким образом, выходные данные дискриминатора должны иметь форму [batch_size, 1] и содержать действительные числа, соответствующие рейтингам каждого входного изображения из мини блока размером `batch_size`.

In [ ]:
def discriminator():
    """
     Вычислить рейтинги на выходе дискриминатора для миниблока входных изображений.
    
     Входы:
     - x: Тензор TensorFlow  сглаженных входных изображений, форма [batch_size, 784]
    
     Возвращает:
     Тензор ТensorFlow  с формой [batch_size, 1], содержащий рейтинги
     принадлежности входных изображений к реальным изображениям.
    """
    model = tf.keras.models.Sequential([
        # Задание: реализовать архитектуру, указанную выше
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
         
        
        #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    ])
    return model


Проверьте на правильность количество параметров дискриминатора

In [ ]:
def test_discriminator(true_count=267009):
    model = discriminator()
    cur_count = count_params(model)
    if cur_count != true_count:
        print('Неправильное количество параметров дискриминатора. {0} вмсето {1}. Проверьте Вашу архитектуру.'.format(cur_count,true_count))
    else:
        print('Ваша архитектура имеет правильное количество параметров дискриминатора.')
        
test_discriminator()

##  Генератор

Теперь построим генератор. Вы должны использовать слои из `tf.keras.layers` для построения модели. Все полносвязанные слои должны содержать смещения. Обратите внимание, что можно использовать модуль tf.nn для доступа к функциям активации. Еще раз, используйте инициализаторы по умолчанию для параметров.

Архитектура генератора:
  * Полносвязанный слой с размером входа tf.shape (z) [1] (количество измерений шума) и размером выхода 1024
  * `ReLU`
  * Полносвязанный слой с выходным размером 1024
  * `ReLU`
  * Полносвязанный слой с выходным размером 784
  * `TanH` (чтобы ограничить выходные значения диапазоном [-1,1])

In [ ]:
def generator(noise_dim=NOISE_DIM):
    """
     Генерирует изображения из случайного вектора шума.
    
     Входы:
     - z: Тензор, содержащий случайный шум с формой [batch_size, noise_dim]
    
     Возвращает:
     Тензор  сгенерированных изображений формы [batch_size, 784].
    """
    model = tf.keras.models.Sequential([
        # Задание: реализовать архитектуру
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
         
        
        #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        
    ])
    return model

Проверьте на правильность число параметров генератора:

In [ ]:
def test_generator(true_count=1858320):
    model = generator(4)
    cur_count = count_params(model)
    if cur_count != true_count:
        print('Неправильное количество параметров генератора. {0} вместо {1}. Проверьте Ващу архитектуру.'.format(cur_count,true_count))
    else:
        print('Ваша архитектура имеет правильное количество параметров генератора.')
        
test_generator()

# Потери GAN


Вычислите потери генератора и дискриминатора. Потери генератора составляют:
$$\ell_G  =  -\mathbb{E}_{z \sim p(z)}\left[\log D(G(z))\right]$$
а потери дискриминатора равны:
$$ \ell_D = -\mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] - \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$

Обратите внимание, что они снижаются, поскольку мы будем «минимизировать» эти потери.

**ПОДСКАЗКА**: Используйте `tf.ones` и `tf.zeros`  для генерации меток для дискриминатора. Используйте `tf.keras.losses.BinaryCrossentropy`  для вычисления функции потерь.

Если ячейка ниже заполнена, то просто ознакомьтесь с кодом.

In [ ]:
def discriminator_loss(logits_real, logits_fake):
    """
     Вычисляет потери дискриминатора, описанные выше.
    
     Входы:
     - logits_real: тензор формы (N, 1), содержащий рейтинги для реальных данных.
     - logits_fake: тензор формы (N, 1), содержащий  рейтинги для поддельных данных.
    
     Возвращает:
     - loss: Тензор, содержащий (скаляр) потери дискриминатора.
    """
    loss = None
    # Задание: определите функцию
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    bce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
    ones = tf.ones_like(logits_real)
    zeros = tf.zeros_like(logits_fake)
    loss = bce(ones, logits_real) + bce(zeros, logits_fake)        
    pass

    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    return loss

def generator_loss(logits_fake):
    """
     Вычисляет потери генератора, описанные выше.

     Входы:
     - logits_fake: Тензор формы (N,), содержащий рейтинги  для поддельных данных.
    
     Возвращает:
     - loss: Тензор , содержащий (скаляр) потери генератора.
    """
    loss = None
    # Задание: определите функцию
    #*****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    bce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
    ones = tf.ones_like(logits_fake)
    loss = bce(ones, logits_fake)        
    pass

    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    
    return loss

Проверьте вычисление потерь GAN. Убедитесь, что потери генератора и дискриминатора вычисляются правильно. Ошибка должна быть менее 1e-8.

In [ ]:
def test_discriminator_loss(logits_real, logits_fake, d_loss_true):
    d_loss = discriminator_loss(tf.constant(logits_real),
                                tf.constant(logits_fake))
    print("Максимум ошибки d_loss: %g"%rel_error(d_loss_true, d_loss))

test_discriminator_loss(answers['logits_real'], answers['logits_fake'],
                        answers['d_loss_true'])

In [ ]:
def test_generator_loss(logits_fake, g_loss_true):
    g_loss = generator_loss(tf.constant(logits_fake))
    print("Максимум ошибки g_loss: %g"%rel_error(g_loss_true, g_loss))

test_generator_loss(answers['logits_fake'], answers['g_loss_true'])

# Оптимизация потерь
Создайте оптимизатор `Adam` со скоростью обучения 1e-3, beta1 = 0,5, чтобы минимизировать G_loss и D_loss по отдельности. Трюк  с уменьшением beta показал свою эффективность для схождения GAN, см. статью [Improved Techniques for Training GANs](https://arxiv.org/abs/1606.03498) . На самом деле, если вы установите для beta1 значение по умолчанию, равное 0,9, есть большая вероятность того, что  потери дискриминатора упадут до нуля, и генератор не сможет полностью обучиться. Фактически, это распространенный режим отказа  GAN: если  D(x) обучается слишком быстро (например, потери приближаются к нулю), то G(z) никогда не сможет обучиться. Часто D(x) обучают на основе SGD с моментом или RMSProp вместо использования Adam , но здесь мы будем использовать Adam  как для D(x), так и для G(z).

In [ ]:
# Задание: создайте оптимизатор Adam для D_solver и G_solver
def get_solvers(learning_rate=1e-3, beta1=0.5):
    """
     Создает оптимизаторы для обучения GAN.
    
     Входы:
     - learning_rate: скорость обучения для обоих решателей
     - beta1: параметр beta1 для обоих решателей (затухание в первого момента)
    
     Возвращает:
     - D_solver: экземпляр tf.optimizers.Adam с корректной скоростью обучения и beta1
     - G_solver: экземпляр tf.optimizers.Adam с корректной скоростью обучения и beta1
    """
    D_solver = None
    G_solver = None
    # Задание: определите функцию
    #*****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
           
    pass

    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    return D_solver, G_solver

# Обучение GAN!
После первой эпохи Вы должны видеть нечеткие контуры, четкие изображения при приближении к эпохе 3 и хорошие изображения, примерно половина из которых будет четкой и отчетливо узнаваемой, при прохождении эпохи 5. В нашем случае мы просто будем обучать D(х) и G(z) на одном миниблоке на каждой итерации. Тем не менее, часто экспериментируют с различными расписаниями обучения D(x) и G(z), иногда делая больше шагов для D(х) или G(z), или даже обучают их по отдельности, пока потери не станут «достаточно хорошими» для одной подсети, а затем переключаются на обучение другой подсети.

In [ ]:
# Гигантская вспомогательная функция
def run_a_gan(D, G, D_solver, G_solver, discriminator_loss, generator_loss,\
              show_every=20, print_every=20, batch_size=128, num_epochs=10, noise_size=96):
    """
     Обучение GAN на определенном количестве эпох.
    
     Входы:
     - D: модель дискриминатора
     - G: модель генератора
     - D_solver: оптимизатор для дискриминатора
     - G_solver: оптимизатор для генератора
     - generator_loss: потери генератора
     - discriminator_loss: потери дискриминатора
     Возвращает: Ничего
    """
    mnist = MNIST(batch_size=batch_size, shuffle=True)
    
    iter_count = 0
    for epoch in range(num_epochs):
        for (x, _) in mnist:
            with tf.GradientTape() as tape:
                real_data = x
                logits_real = D(preprocess_img(real_data))

                g_fake_seed = sample_noise(batch_size, noise_size)
                fake_images = G(g_fake_seed)
                logits_fake = D(tf.reshape(fake_images, [batch_size, 784]))

                d_total_error = discriminator_loss(logits_real, logits_fake)
                d_gradients = tape.gradient(d_total_error, D.trainable_variables)      
                D_solver.apply_gradients(zip(d_gradients, D.trainable_variables))
            
            with tf.GradientTape() as tape:
                g_fake_seed = sample_noise(batch_size, noise_size)
                fake_images = G(g_fake_seed)

                gen_logits_fake = D(tf.reshape(fake_images, [batch_size, 784]))
                g_error = generator_loss(gen_logits_fake)
                g_gradients = tape.gradient(g_error, G.trainable_variables)      
                G_solver.apply_gradients(zip(g_gradients, G.trainable_variables))

            if (iter_count % show_every == 0):
                print('Эпоха: {}, Итерация: {}, D: {:.4}, G:{:.4}'.format(epoch, iter_count,d_total_error,g_error))
                imgs_numpy = fake_images.cpu().numpy()
                show_images(imgs_numpy[0:16])
                plt.show()
            iter_count += 1
    
    # случайный шум, подаваемый на вход генератора
    z = sample_noise(batch_size, noise_size)
    # сгенерированное изображение
    G_sample = G(z)
    print('Конечные изображения')
    show_images(G_sample[:16])
    plt.show()

#### Обучите Вашу GAN. Это займет 10 минут на CPU или около 2 минут на GPU.



In [ ]:
# Создание дискриминатора
D = discriminator()

# Создание генератора
G = generator()

# Используйте функцию, которую вы написали ранее, чтобы получить оптимизаторы для  дискриминатора и генератора
D_solver, G_solver = get_solvers()

# Запуск!
run_a_gan(D, G, D_solver, G_solver, discriminator_loss, generator_loss)

# GAN на основе метода наименьших квадратов
Теперь мы рассмотрим [GAN на основе наименьших квадратов](https://arxiv.org/abs/1611.04076) - более новую, более стабильную альтернативу исходной функции потерь GAN. Для этой части все, что нам нужно сделать, это изменить функцию потерь и переобучить модель. Мы реализуем уравнение (9) из статьи. Потери генератора:
$$\ell_G  =  \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[\left(D(G(z))-1\right)^2\right]$$
Потери дискриминатора:
$$ \ell_D = \frac{1}{2}\mathbb{E}_{x \sim p_\text{data}}\left[\left(D(x)-1\right)^2\right] + \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[ \left(D(G(z))\right)^2\right]$$

**СОВЕТЫ**: Вместо того, чтобы вычислять математическое ожидание, будем выполнять усреднение по элементам мини-блока, поэтому убедитесь, что Вы вычисляете потери путем усреднения, а не суммирования. При подключении к $D(x)$ и $D(G(z))$ используйте прямой выход дискриминатора (`score_real` и` score_fake`).

Если ячейка ниже заполнена, то просто ознакомьтесь с кодом.

In [ ]:
def ls_discriminator_loss(scores_real, scores_fake):
    """
     Вычисляет потери наименьших квадратов для дискриминатора GAN .
     Входы:
     - scores_real: тензор формы (N, 1), содержащий рейтинги для реальных данных.
     - scores_fake: тензор формы (N, 1), содержащий рейтинги для поддельных данных.
    
     Выходы:
     - loss: Тензор, содержащий потери
    
     """
    loss = None
    # Задание: определите функцию
    #*****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    loss = 0.5*tf.reduce_mean(tf.square(scores_real - 1)) + 0.5*tf.reduce_mean(tf.square(scores_fake))        
    pass

    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    return loss

def ls_generator_loss(scores_fake):
    """
     Вычисляет потери  наименьших квадратов для генератора GAN.
    
     Входы:
     - scores_fake: тензор формы (N, 1), содержащий рейтинги для поддельных данных.
    
     Выходы:
     - loss: Тензор, содержащий потери.
    """
    loss = None
    
    #*****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    loss = 0.5*tf.reduce_mean((tf.square(scores_fake-1)))        
    pass

    #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    return loss

Проверьте LSGAN потери. Ошибки должны быть менее 1e-8.

In [ ]:
def test_lsgan_loss(score_real, score_fake, d_loss_true, g_loss_true):
    
    d_loss = ls_discriminator_loss(tf.constant(score_real), tf.constant(score_fake))
    g_loss = ls_generator_loss(tf.constant(score_fake))
    print("Максимальная ошибка d_loss: %g"%rel_error(d_loss_true, d_loss))
    print("Максимальная ошибка g_loss: %g"%rel_error(g_loss_true, g_loss))

test_lsgan_loss(answers['logits_real'], answers['logits_fake'],
                answers['d_loss_lsgan_true'], answers['g_loss_lsgan_true'])

Создайте новые обучающие шаги, так чтобы минимизировать LSGAN потери:


In [ ]:
# Создание дискриминатора
D = discriminator()

# Создание генератора
G = generator()

# Используйте функцию, которую вы написали ранее, чтобы получить оптимизаторы для  дискриминатора и генератора
D_solver, G_solver = get_solvers()

# Запуск!
run_a_gan(D, G, D_solver, G_solver, ls_discriminator_loss, ls_generator_loss)

# Глубокие сверточные GAN сети

В первой части блокнота мы реализовали почти прямую копию оригинальной сети GAN  Яна Гудфеллоу. Однако эта сетевая архитектура не обеспечивает понимание реальных пространственных отношений. Она не может делать общих заключений о таких составляющих изображений, как «края», потому что в ней отсутствуют какие-либо сверточные слои. В этом разделе мы реализуем некоторые идеи из [DCGAN](https://arxiv.org/abs/1511.06434), где  используются сверточные сети в качестве дискриминаторов и генераторов.

#### Дискриминатор
Будем использовать дискриминатор, инспирированный  классификатором TensorFlow MNIST,который способен довольно быстро получить точность на данных MNIST выше 99%. *Обязательно проверьте размеры x и измените их при необходимости*, полносвязанные слои принимают [N, D] тензоры, в то время как блоки conv2d принимают  [N, H, W, C] тензоры. Используйте `tf.keras.layers`, чтобы определить следующую архитектуру:

* Conv2D: 32 Фильтра  5x5, Stride 1, padding 0
* Leaky ReLU(alpha=0.01)
* Max Pool 2x2, Stride 2
* Conv2D: 64 Фильтра 5x5, Stride 1, padding 0
* Leaky ReLU(alpha=0.01)
* Max Pool 2x2, Stride 2
* Уплощение (Flatten)
* Полносвязанный слой с размером выхода 4 x 4 x 64
* Leaky ReLU(alpha=0.01)
* Полносвязанный слой с размером выхода 1


Используйте смещения для всех сверточных и полносвязанных слоев и используйте инициализаторы параметров по умолчанию. Обратите внимание, что дополнение 0 может быть выполнено с помощью параметра padding = 'VALID'.

In [ ]:
def discriminator():
    """
     Вычисляет рейтинг дискриминатора для миниблока входных изображений.
    
     Входы:
     - x: Тензор TensorFlow  сглаженных входных изображений, форма [batch_size, 784]
    
     Возвращает:
     Тензор TensorFlow  с формой [batch_size, 1], содержащий рейтинги
     принадлежности входных изображений к реальным.
    """
    model = tf.keras.models.Sequential([
        # Задание: реализовать архитектуру
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
       
        
        #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    ])
    return model

model = discriminator()
test_discriminator(1102721)

#### Генератор
Для генератора мы скопируем архитектуру из статьи [InfoGAN](https://arxiv.org/pdf/1606.03657.pdf). См. Приложение C.1 MNIST. Используйте `tf.keras.layers`  для своей реализации. Будет полезно знакомство с `tf.keras.layers.Conv2DTranspose`. Архитектура выглядит следующим образом:


* Полносвязанный слой с размером выхода 1024 
* `ReLU`
* Блочная нормализация (batchNorm)
* Полносвязанный слой с размером выхода 7 x 7 x 128 
* `ReLU`
* Блочная нормализация (batchNorm)
* Приведение размера тензора изображения к 7, 7, 128
* Conv2DTranspose: 64 фильтра 4x4, stride 2
* `ReLU`
* Блочная нормализация (batchNorm)
* Conv2DTranspose: 1 фильтр of 4x4, stride 2
* `TanH`

Используйте смещения для полносвязанных и транспонирующих сверточных слоев. Используйте инициализаторы по умолчанию для  параметров. Для параметра padding выберите значение same в случае транспонированной свертки. Для  блочной нормализации предполагаем, что она выполняется всегда в режиме «обучения».

In [ ]:
def generator(noise_dim=NOISE_DIM):
    """
     Генерация изображений из случайного вектора шума.
    
     Входы:
     - z: Тензор TensorFlow случайного шума с формой [batch_size, noise_dim]
    
     Возвращает:
      Тензор TensorFlow  сгенерированных изображений с формой [batch_size, 784].
    """
    model = tf.keras.models.Sequential([
        # Задание: реализовать архитектуру
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
          
        ])
        #*****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    return model
test_generator(6595521)


Мы должны воссоздать нашу сеть, так как мы изменили наши функции.


###  Обучение и оценка DCGAN
Это та часть задания, которая значительно выигрывает от использования графического процессора. Требуется 3 минуты на GPU для запрошенных пяти эпох. Или около 50 минут на двухъядерном ноутбуке с CPU (используйте именно 3 эпохи, если вы выполняете задачу на CPU).

In [ ]:
# Создание дискриминатора
D = discriminator()

# Создание генератора
G = generator()

# Используйте функцию, которую вы написали ранее, чтобы получить оптимизаторы для  дискриминатора и генератора
D_solver, G_solver = get_solvers()

# Запуск!
run_a_gan(D, G, D_solver, G_solver, discriminator_loss, generator_loss, num_epochs=3)

## Вопрос 1

Рассмотрим пример, чтобы понять, почему альтернативная минимизация одной и той же цели (как в GAN) может быть непростым делом.

Рассмотрим $f(x,y)=xy$. Что оценивает следующая мимнимаксная функция $\min_x\max_y f(x,y)$? (Подсказка: minmax пытается минимизировать максимально достижимое значение.)

Теперь попробуйте оценить эту функцию численно за 6 шагов, начиная с точки $(1,1)$,
и попеременно используя градиенты переменных (сначала обновляя y, затем обновляя x с использованием этого обновленного y) с размером шага $1$. **Здесь размер шага - скорость обучения, а  сами обновления будут равны: 
скорость_обучения * градиент**.
Будет полезным записать шаг обновления в терминах $x_t,y_t,x_{t+1},y_{t+1}$ .

Кратко объясните, что оценивает $\min_x\max_y f(x,y)$, и запишите шесть пар явных значений для $ (x_t, y_t) $ в таблице ниже.

### Ваши ответы:
 
 $y_0$ | $y_1$ | $y_2$ | $y_3$ | $y_4$ | $y_5$ | $y_6$ 
 ----- | ----- | ----- | ----- | ----- | ----- | ----- 
   1   |       |       |       |       |       |       
 $x_0$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $x_6$ 
   1   |       |       |       |       |       |       
   


## Вопрос 2

Используя этот метод, мы когда-нибудь достигнем оптимального значения? Почему да или почему нет?

### Ваш ответ:

## Вопрос 3
Если потери генератора уменьшаются во время обучения, в то время как потери дискриминатора с самого начала остаются на постоянном высоком значении, это хороший знак? Почему да или почему нет? Достаточно качественного ответа. 
### Ваш ответ:

